Get data from github

In [1]:
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/train.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/valid.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/test.csv
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_classifier.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/bert_dataset.py
!wget https://raw.githubusercontent.com/shitkov/bert4classification/main/requirements.txt

--2021-07-05 14:35:37--  https://raw.githubusercontent.com/shitkov/bert4classification/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9340188 (8.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   8.91M  --.-KB/s    in 0.04s   

2021-07-05 14:35:38 (199 MB/s) - ‘train.csv’ saved [9340188/9340188]

--2021-07-05 14:35:38--  https://raw.githubusercontent.com/shitkov/bert4classification/main/valid.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 518234 (506K) [text/plain]
Saving

Installing Packages

In [2]:
!pip install -r /content/requirements.txt

     |████████████████████████████████| 804.1MB 23kB/s 
     |████████████████████████████████| 1.2MB 45.0MB/s 
     |████████████████████████████████| 2.1MB 42.0MB/s 
     |████████████████████████████████| 7.1MB 21.6MB/s 
     |████████████████████████████████| 3.3MB 51.3MB/s 
     |████████████████████████████████| 901kB 48.1MB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.1 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [3]:
import pandas as pd

In [4]:
train_data = pd.read_csv('/content/train.csv')
valid_data = pd.read_csv('/content/valid.csv')
test_data  = pd.read_csv('/content/test.csv')

In [5]:
from bert_dataset import CustomDataset
from bert_classifier import BertClassifier

Initialize BERT classifier

In [6]:
classifier = BertClassifier(
        model_path='cointegrated/rubert-tiny',
        tokenizer_path='cointegrated/rubert-tiny',
        n_classes=2,
        epochs=2,
        model_save_path='/content/bert.pt'
)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not i

Prepare data and helpers for train and evlauation

In [7]:
classifier.preparation(
        X_train=list(train_data['text']),
        y_train=list(train_data['label']),
        X_valid=list(valid_data['text']),
        y_valid=list(valid_data['label'])
    )

Train loop

In [8]:
classifier.train()

Epoch 1/2
Train loss 0.7945727040289298 accuracy 0.6834731168148825
Val loss 0.8699987335717541 accuracy 0.7110332749562172
----------
Epoch 2/2
Train loss 0.8046949874838054 accuracy 0.7415205588744672
Val loss 0.9018999434438866 accuracy 0.717338003502627
----------


Check test data

In [9]:
texts = list(test_data['text'])
labels = list(test_data['label'])

predictions = [classifier.predict(t) for t in texts]

In [10]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro')[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.7257295950143019, recall: 0.7188798418940976, f1score: 0.7204235791808031
